In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse

In [2]:
df_train = pd.read_csv(r'..\...\treino\r2_ig_train_statements.csv', sep=';')
df_train = df_train.drop(df_train[df_train.Polarity =='other'].index)
df_train.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Gente acabei de descobrir que eu sou batizado ...,r2_ig,0,no,r2_ig_733,_1074823139094392832,1100,"[('gente', 'N'), ('acabei', 'V'), ('de', 'PREP...",[],"{'igreja': 1, 'gente': 1, 'tio': 1, 'servo': 1..."
1,Hoje tá fazendo um mês desde a minha consagraç...,r2_ig,1,no,r2_ig_504,_1249494987290664961,3653,"[('hoje', 'ADV'), ('tá', 'V'), ('fazendo', 'V'...",[],"{'saudades': 1, 'igreja': 1, 'analu': 1, 'vida..."
2,depois de 5 anos vou pisar na igreja novamente...,r2_ig,0,no,r2_ig_182,_1219761054089207808,1398,"[('depois', 'PREP'), ('de', 'PREP'), ('5', 'NU...",[],"{'q': 2, 'igreja': 1, 'anos': 1, 'priscila': 1}"
3,Ser de tudo certo domingo eu vou p igreja dnv,r2_ig,1,no,r2_ig_1398,_1138161702015578112,384,"[('ser', 'V'), ('de', 'PREP'), ('tudo', 'PROAD...",[],"{'igreja': 1, 'p': 1, 'domingo': 1}"
4,A mentalidade de cliente ainda é vista entre f...,r2_ig,1,no,r2_ig_1648,_1235937222060388352,1207,"[('a', 'ART'), ('mentalidade', 'N'), ('de', 'P...",[],"{'pessoas': 1, 'frequentadores': 1, 'boates': ..."
...,...,...,...,...,...,...,...,...,...,...
5347,A igreja mais bonita somos nós,r2_ig,1,no,r2_ig_1201,_1310334653069492224,665,"[('a', 'ART'), ('igreja', 'N'), ('mais', 'ADV'...",[],{'igreja': 1}
5348,Ir pra igreja é mt bom,r2_ig,1,no,r2_ig_340,_975451071803023360,1889,"[('ir', 'V'), ('pra', 'PREP+ART'), ('igreja', ...",[],"{'igreja': 1, 'mt': 1}"
5349,terra nas maos de poucos é um absurdo que só a...,r2_ig,0,no,r2_ig_142,_5241457534,16,"[('@', 'NUM'), ('caiodezorzi', 'N'), ('terra',...",['caiodezorzi'],"{'bençaos': 1, 'caiodezorzi': 1, 'igreja': 1, ..."
5350,hoje vou pedir pro meu pai pra passar pela fre...,r2_ig,1,no,r2_ig_699,_1271450299593633792,938,"[('hoje', 'ADV'), ('vou', 'V'), ('pedir', 'V')...",[],"{'gratidão': 2, 'igreja': 1, 'pe': 1, 'pai': 1..."


In [3]:
df_test = pd.read_csv(r'..\...\Teste\r2_ig_test_statements.csv', sep=';')
df_test = df_test.drop(df_test[df_test.Polarity =='other'].index)
df_test.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Minha cara racha com esses esquenta banco de i...,r2_ig,0,no,r2_ig_1060,_780988283614531584,2347,"[('minha', 'PROADJ'), ('cara', 'N'), ('racha',...",[],"{'igreja': 1, 'testemunho': 1, 'irmãzinha': 1,..."
1,Vocês falam da igreja ter mt dinheiro e eu n e...,r2_ig,0,no,r2_ig_1266,_680416366814429184,867,"[('vocês', 'PROPESS'), ('falam', 'V'), ('da', ...",[],"{'igreja': 1, 'dinheiro': 1, 'conversa': 1, 't..."
2,não apoiar não faz da igreja homofobica e sim ...,r2_ig,1,no,r2_ig_874,_1094251009000570880,659,"[('não', 'ADV'), ('apoiar', 'V'), ('não', 'ADV...",[],"{'princípios': 1, 'deus': 1, 'igreja': 1, 'rou..."
3,"a igreja tem 60 pessoas, tem 50 cantando direi...",r2_ig,1,no,r2_ig_532,_1060297965196197889,2319,"[('parece', 'V'), ('que', 'KS'), ('é', 'V'), (...",[],"{'igreja': 1, 'metros': 1, 'batalha': 1, 'lado..."
4,"Será nesse sábado e domingo, venha visitar a i...",r2_ig,1,no,r2_ig_132,_1050738081710497793,325,"[('será', 'V'), ('nesse', 'PREP+PROADJ'), ('sá...",['mission'],"{'deus': 1, 'igreja': 1, 'mission': 1, 'sábado..."
...,...,...,...,...,...,...,...,...,...,...
1780,é q ele nao se pronunciou sobre a queima de ig...,r2_ig,1,no,r2_ig_115,_1318536529934012416,1290,"[('@', 'NUM'), ('januario_voj', 'N'), ('é', 'V...",['januario_voj'],"{'q': 1, 'safados': 1, 'queima': 1, 'igrejas':..."
1781,E nítido o ódio a quem e cristão no nosso país...,r2_ig,1,no,r2_ig_702,_1120019454900015104,1741,"[('e', 'KC'), ('nítido', 'PCP'), ('o', 'ART'),...",[],"{'país': 1, 'igreja': 1, 'momento': 1, 'ódio':..."
1782,Cota pra conteúdo gospel que já conta com um c...,r2_ig,0,no,r2_ig_1395,_1136028798644903938,2843,"[('cota', 'N'), ('pra', 'PREP'), ('conteúdo', ...",[],"{'conteúdo': 1, 'dinheiro': 1, 'igrejas': 1, '..."
1783,tá amarrado e repreendido senhor Jesus vai que...,r2_ig,1,no,r2_ig_1114,_956208092131352576,303,"[('tá', 'V'), ('amarrado', 'PCP'), ('e', 'KC')...",[],"{'jesus': 2, 'igreja': 1, 'obra': 1, 'maldição..."


In [4]:
#carregando unlabeled data
df_u = pd.read_csv(r'..\...\Unlabeled\igreja.csv')
#df_u = df.sample(frac=1).reset_index(drop=True)
df_u

,Unnamed: 0,Text
0,0,Quem diz que é normal nunca nem entrou em uma ...
1,1,"evangélicos"" até saírem da igreja e virarem a ..."
2,2,"Kkkkkkkkkkkk essa daí e profissional , e da ig..."
3,3,Minha mãe chegou da igreja agora é disse que f...
4,4,Ahahahahaha mds labaxurias é um linguajar estr...
...,...,...
34585,34585,Sou mt princesa do senhor com fotinha no face ...
34586,34586,Quando eu tinha que fzr as coisas na igreja
34587,34587,"Vamos sair"" ""Vamos, mas primeiro vou pra igreja"
34588,34588,Wi-Fi daqui da igreja ta melhor que o la de casa


In [5]:
rows_train = len(df_train.index)
rows_test = len(df_test.index)
rows_label=rows_train+rows_test

In [6]:
rows_unlabel = len(df_u.index)
rows_unlabel

34590

In [7]:
frames = [df_train.Text, df_test.Text, df_u.Text]
X = pd.concat(frames)
X

0        Gente acabei de descobrir que eu sou batizado ...
1        Hoje tá fazendo um mês desde a minha consagraç...
2        depois de 5 anos vou pisar na igreja novamente...
3            Ser de tudo certo domingo eu vou p igreja dnv
4        A mentalidade de cliente ainda é vista entre f...
                               ...                        
34585    Sou mt princesa do senhor com fotinha no face ...
34586          Quando eu tinha que fzr as coisas na igreja
34587      Vamos sair" "Vamos, mas primeiro vou pra igreja
34588     Wi-Fi daqui da igreja ta melhor que o la de casa
34589    Mandei msgn para Yasmin e ela ta na igreja com...
Name: Text, Length: 41727, dtype: object

In [8]:
#vetoriza conjunto completo
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer( )
X_counts = count_vect.fit_transform(X)
X_counts.shape

(41727, 40403)

In [9]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_counts)
X_tf = tf_transformer.transform(X_counts)
X_tf.shape
X_ori = X_tf
X_ori.shape

(41727, 40403)

In [10]:
x_train = X_ori[0:rows_train,:]
x_test = X_ori[rows_train:rows_train+rows_test,:]
x_test.shape


(1785, 40403)

In [11]:
#y_ori = df.Polarity
y_train = df_train.Polarity
y_test = df_test.Polarity
y_test.shape

(1785,)

In [12]:
#classificação
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
model = LogisticRegression(random_state=0)
model.fit(x_train,y_train)
pred_values = model.predict(x_test)     
acc = accuracy_score(pred_values , y_test)
acc

0.7680672268907563

In [13]:
f1_score(y_test, pred_values, average='macro')

0.7680636600112487

In [14]:
precision_score(y_test, pred_values, average='macro')

0.7682062821939207

In [15]:
recall_score(y_test, pred_values, average='macro')

0.7681544256120527

In [16]:
import eli5
classes = [0,1]
weights = eli5.show_weights(model, feature_names = count_vect.get_feature_names(),  target_names=classes, top = 50)   # ['diag','ctrl'] ???
url = "teste.htm"
with open(url,'wb') as f:
    f.write(weights.data.encode("UTF-8"))

C:\Users\camil\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
weights

Weight?,Feature
+2.989,vou
+2.527,ir
+2.506,saudade
+2.409,nossa
+2.259,amanhã
+2.185,hoje
+1.999,maria
+1.952,fui
+1.857,indo
+1.843,senhor


In [172]:
#predição dos não-rotulados
X_label = X_ori[rows_label:rows_label+ rows_unlabel]
pred_values_label= model.predict(X_label) 

In [173]:
#avalia a probabilidade de estar certo
prob_label = model.predict_proba(X_label) 


In [174]:
#seleciona os itens que vão entrar no conjunto de treino
threshold = 0.01
t_size = int(threshold*rows_unlabel) #quanto vai adicionar a cada iteração
rows_train = x_train.shape[0]
rows_train

5352

In [175]:
reliable_pred_favor, reliable_pred_against  = [], []
for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
    #se 1, inclui no favor; senão, no against
    #incluir id, probabilidade e valor predito (1 ou 0)
    (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    

In [176]:
#coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)

In [177]:
y= np.asarray(y_train)
index_list=[x for x in range(0, rows_train)] #indices para o novo X a partir do X original
iteractions = 0
position = 0
index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

while iteractions < 0.99:
    iteractions = iteractions + threshold
    for i in range(int(t_size/2)):
        #pega um a favor
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 1) #empilha o y
        index_labels.remove(index+rows_label)
        #pega um contra
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 0) #empilha o y
        index_labels.remove(index+rows_label)
        #anda uma posição nos dois 
        position+=1

    X_train_novo = X_ori[index_list,:]#atualiza o X rotulado com os índices que estão em index_list
    X_label = X_ori[index_labels,:] #pega só os índices que sobraram
    rows_unlabel = X_label.get_shape()[0] #novo número de linhas sem rótulo
    rows_label = X_train_novo.get_shape()[0] + rows_test #linhas com rótulo novas + as de teste que estão guardadas
    X_ori = sparse.vstack([X_train_novo, x_test, X_label]) #empilha treino, teste e unlabelled
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)] #atualiza os unlabelled
    position = 0

    model_novo = LogisticRegression(random_state=0)
    model_novo.fit(X_train_novo,y)
    #testa nos conjuntos originais
    pred_values_novo = model_novo.predict(x_test)     
    acc = accuracy_score(pred_values_novo , y_test)
    print("iteraction:",iteractions, "accuracy:", acc)
    
    pred_values_label= model_novo.predict(X_label) 
    prob_label = model.predict_proba(X_label) 
    reliable_pred_favor, reliable_pred_against  = [], []
    for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
        (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    
    reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
    reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)    
    
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

iteraction: 0.01 accuracy: 0.7675070028011205
iteraction: 0.02 accuracy: 0.7714285714285715
iteraction: 0.03 accuracy: 0.7703081232492998
iteraction: 0.04 accuracy: 0.7697478991596639
iteraction: 0.05 accuracy: 0.7624649859943977
iteraction: 0.060000000000000005 accuracy: 0.7658263305322129
iteraction: 0.07 accuracy: 0.7591036414565826
iteraction: 0.08 accuracy: 0.7663865546218488
iteraction: 0.09 accuracy: 0.7602240896358543
iteraction: 0.09999999999999999 accuracy: 0.769187675070028
iteraction: 0.10999999999999999 accuracy: 0.7579831932773109
iteraction: 0.11999999999999998 accuracy: 0.7680672268907563
iteraction: 0.12999999999999998 accuracy: 0.7663865546218488
iteraction: 0.13999999999999999 accuracy: 0.7703081232492998
iteraction: 0.15 accuracy: 0.7658263305322129
iteraction: 0.16 accuracy: 0.7647058823529411
iteraction: 0.17 accuracy: 0.7518207282913165
iteraction: 0.18000000000000002 accuracy: 0.7607843137254902
iteraction: 0.19000000000000003 accuracy: 0.7602240896358543
iterac

C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.21000000000000005 accuracy: 0.7635854341736694
iteraction: 0.22000000000000006 accuracy: 0.7563025210084033
iteraction: 0.23000000000000007 accuracy: 0.7697478991596639
iteraction: 0.24000000000000007 accuracy: 0.7579831932773109
iteraction: 0.25000000000000006 accuracy: 0.7647058823529411


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.26000000000000006 accuracy: 0.7602240896358543
iteraction: 0.2700000000000001 accuracy: 0.7680672268907563


...

C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.9700000000000006 accuracy: 0.684593837535014


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.9800000000000006 accuracy: 0.6515406162464986
iteraction: 0.9900000000000007 accuracy: 0.6644257703081232


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


33902